In [1]:
from pathlib import Path
import pickle
import pandas as pd
import numpy as np

f_data = Path("/Users/bnowacki/Documents/Git Repositories/fine-tuning-for-rapid-soh-estimation/processed_data/UConn-ILCC-NMC/data_slowpulse_1.pkl") 
data = pickle.load(open(f_data, 'rb'))

print(data['cell_id'].shape, data['group_id'].shape, data['voltage'].shape, data['soh'].shape)
data.keys()

(24048,) (24048,) (24048, 101) (24048,)


/var/folders/21/fsx4ddjs3fg2wgpl7_ksh0k00000gn/T/ipykernel_22953/2191965955.py:7: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.load(open(f_data, 'rb'))


dict_keys(['cell_id', 'group_id', 'rpt', 'num_cycles', 'soc', 'soc - coulomb', 'pulse_type', 'voltage', 'q_dchg', 'soh', 'dcir_chg_10', 'dcir_dchg_10', 'dcir_chg_20', 'dcir_dchg_20', 'dcir_chg_30', 'dcir_dchg_30', 'dcir_chg_40', 'dcir_dchg_40', 'dcir_chg_50', 'dcir_dchg_50', 'dcir_chg_60', 'dcir_dchg_60', 'dcir_chg_70', 'dcir_dchg_70', 'dcir_chg_80', 'dcir_dchg_80', 'dcir_chg_90', 'dcir_dchg_90'])

In [2]:
from modularml.core.data_structures.feature_set import FeatureSet
from modularml.core.data_structures.feature_subset import FeatureSubset

fs = FeatureSet.from_dict(
	label='MyFeatureSet',
	data={
		'voltage': data['voltage'],
		'soh': data['soh'],
		'cell_id': data['cell_id'],
  		'group_id': data['group_id'],
    	'pulse_type': data['pulse_type'],
     	'pulse_soc': data['soc'],
	},
	feature_keys='voltage',
	target_keys='soh',
	tag_keys=['cell_id', 'group_id', 'pulse_type', 'pulse_soc']
)
print(fs)
print(fs.list_tags())

FeatureSet(label='MyFeatureSet', n_samples=24048)
['__sample_id__', '__sample_index__', 'cell_id', 'group_id', 'pulse_type', 'pulse_soc']


In [3]:
fss = fs.filter(cell_id=[1,2], pulse_type='chg', pulse_soc=[10,20,])
fss

FeatureSubset(label='filtered', n_samples=128)

In [4]:
# from matplotlib import pyplot as plt

# fs_vs = fs.get_all_features(format='dict')['voltage']
# fss_vs = fss.get_all_features(format='dict')['voltage']

# for v in fs_vs:
#     plt.plot(v, '-', color='C0')
    
# for v in fss_vs:
#     plt.plot(v, '-', color='C1')
    
# plt.show()

In [5]:
from modularml.core.splitters.random_splitter import RandomSplitter

print(fs)
fs.clear_subsets()
fs.split_random(ratios={'train': 0.6, 'val':0.2, 'test':0.2})
# fs.split_by_condition(
# 	cond_train= {'cell_id':(lambda x: x < 8)},
# 	cond_val= {'cell_id':(lambda x: (x >= 8) & (x < 36))},
# 	cond_test= {'cell_id':(lambda x: (x >= 36) & (x <= 44))},
# )
print(fs.available_subsets)

train = fs.get_subset(name='train')
print(train)

print(fs.available_subsets)

FeatureSet(label='MyFeatureSet', n_samples=24048)
dict_keys(['train', 'val', 'test'])
FeatureSubset(label='train', n_samples=14428)
dict_keys(['train', 'val', 'test'])


### Testing FeatureSamplers

In [6]:
from modularml.core.samplers.feature_sampler import FeatureSampler


sampler = FeatureSampler(
	source=fs.get_subset('train'),
	batch_size=8, 
 	shuffle=False,
  	drop_last=True
)

print(len(sampler), len(fs.get_subset('train')))

1803 14428


In [7]:
batch = sampler[0]
batch.formatted_samples['main']

[Sample(id=06ecb5f7..., features={'voltage': 'ndarray'}, targets={'soh': 'float64'}, tags={'cell_id': 'int64', 'group_id': 'int64', 'pulse_type': 'str_', 'pulse_soc': 'int64'}, index=4812),
 Sample(id=17e85359..., features={'voltage': 'ndarray'}, targets={'soh': 'float64'}, tags={'cell_id': 'int64', 'group_id': 'int64', 'pulse_type': 'str_', 'pulse_soc': 'int64'}, index=12812),
 Sample(id=33d4fabc..., features={'voltage': 'ndarray'}, targets={'soh': 'float64'}, tags={'cell_id': 'int64', 'group_id': 'int64', 'pulse_type': 'str_', 'pulse_soc': 'int64'}, index=665),
 Sample(id=f0b22084..., features={'voltage': 'ndarray'}, targets={'soh': 'float64'}, tags={'cell_id': 'int64', 'group_id': 'int64', 'pulse_type': 'str_', 'pulse_soc': 'int64'}, index=867),
 Sample(id=8f4a5b1f..., features={'voltage': 'ndarray'}, targets={'soh': 'float64'}, tags={'cell_id': 'int64', 'group_id': 'int64', 'pulse_type': 'str_', 'pulse_soc': 'int64'}, index=2419),
 Sample(id=ffc2726f..., features={'voltage': 'ndarr

In [8]:
batch = sampler[0]
print(batch.feature_shape, batch.target_shape)
batch._sample_weights

(8, 1, 101) (8, 1)


{'main': [1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
from modularml.core.data_structures.batch import BatchComponentSelector

selection = BatchComponentSelector(
	role='main',
	sample_attribute='tags',
	attribute_key='cell_id'
)
features = batch.select(selection, format='df')
# print(features.shape)
features


,main.tags.cell_id
0,10
1,22
2,2
3,2
4,5
5,29
6,2
7,17


In [9]:
print("Available roles: ", sampler.available_roles)
features, targets, tags = sampler[0].unpack(format='df')['main']
tags

Available roles:  ['main']


,_sample_id_,_sample_index_,cell_id,group_id,pulse_type,pulse_soc
0,04f542c2-4d53-41b9-9cdd-7f4537522dd3,4812,10,3,chg,40
1,c63aa493-06c3-468c-aded-5ed7f74cd978,12812,22,6,chg,80
2,9c8c64b4-c697-4ef2-8fb8-0286bc58d967,665,2,1,dchg,90
3,2b516cf4-daad-4be2-bb7b-3189813615a0,867,2,1,dchg,20
4,7c4caba9-34b2-487c-beda-09cac47e688c,2419,5,2,dchg,40
5,0eb90d7b-0322-4b6d-b2a3-c032cd34a534,15985,29,8,dchg,10
6,a5f06139-98a1-427c-8e4a-02f08a164b1f,1021,2,1,dchg,70
7,76f53eef-4ee4-4493-9045-78505a90e477,8613,17,5,dchg,50


In [10]:
# Test stratified sampling
sampler_strat = FeatureSampler(
	source=fs,
 	batch_size=9,
	shuffle=True,
    stratify_by=['pulse_soc', ],
    drop_last=True,
)
print(len(sampler_strat))
roles = sampler_strat.available_roles
for b in sampler_strat:
    features, targets, tags = b.unpack(format='df')[roles[0]]
    assert len(tags['pulse_soc'].unique()) == 9
    
features, targets, tags = sampler_strat[0].unpack(format='df')[roles[0]]
tags

2672


,_sample_id_,_sample_index_,cell_id,group_id,pulse_type,pulse_soc
0,a59867a7-31ab-45c8-99e9-c7ab9509e3ed,16650,30,8,chg,10
1,b098dccf-7753-4494-9f76-ce92fa16fed5,3315,7,2,dchg,20
2,3bee9036-310b-4445-8e77-8a82ea1e6b3b,12263,21,6,dchg,30
3,dec6bb45-4fda-4822-8bef-6131d419332a,3679,8,2,dchg,40
4,6334c01b-f43e-4d0a-8c02-b49fa7454299,1341,3,1,dchg,50
5,e3a3ff2c-e16b-4032-803f-d3c54e825b14,11783,20,5,dchg,60
6,a6d8cf03-e505-4b32-9d8a-be8ece60e18b,2496,5,2,chg,70
7,6fe6178a-a744-4145-91cd-36f82325e346,4298,9,3,chg,80
8,73977171-cb30-489d-bf7c-803f965d3550,8585,17,5,dchg,90


In [11]:
# Test grouped sampling
sampler_grouped = FeatureSampler(
	source=fs,
 	batch_size=8,
	shuffle=True,
    group_by=['pulse_soc', ],
    drop_last=True,
)
print(len(sampler_grouped))
roles = sampler_grouped.available_roles
for b in sampler_grouped:
    features, targets, tags = b.unpack(format='df')[roles[0]]
    assert len(tags['pulse_soc'].unique()) == 1
    
features, targets, tags = sampler_grouped[0].unpack(format='df')[roles[0]]
tags

3006


,_sample_id_,_sample_index_,cell_id,group_id,pulse_type,pulse_soc
0,58a03d8f-bf5f-4c89-8b0f-15a2fefb005a,23709,44,11,dchg,20
1,6f258ad8-4279-4a05-9c05-067cdd8ef95c,17769,31,8,dchg,20
2,25e0ed49-2db3-4968-87e8-0ac89137dd12,2775,6,2,dchg,20
3,8ccb9167-c5f5-44de-bdd1-9ce2153fc1b6,4538,9,3,chg,20
4,53363ffe-8824-47a2-8f52-651e550e136c,21027,38,10,dchg,20
5,842d40ee-443e-4de5-8ed6-4e5d72d822b1,7995,16,4,dchg,20
6,d6db38d4-bdde-437c-b910-057b08d14ceb,11018,19,5,chg,20
7,70baffb8-414e-4949-8bd4-8c90ab86ae34,11522,20,5,chg,20


In [12]:
# Test grouped sampling
sampler_grouped = FeatureSampler(
	source=fs,
 	batch_size=8,
	shuffle=True,
    group_by=['pulse_type', 'cell_id', 'pulse_soc'],
    drop_last=True,
)
print(len(sampler_grouped))
roles = sampler_grouped.available_roles
features, targets, tags = sampler_grouped[0].unpack(format='df')[roles[0]]
tags

2628


,_sample_id_,_sample_index_,cell_id,group_id,pulse_type,pulse_soc
0,e70029ed-203d-4951-89bb-a5deb969dea9,22768,42,11,chg,90
1,ce7fbe0f-02e5-4738-9ca1-5ecdbc925cfc,22426,42,11,chg,90
2,6945aeb7-5371-48e9-949e-c916ccb55165,22390,42,11,chg,90
3,a922c490-c2b4-4e91-84ed-3a55f66c94b5,22372,42,11,chg,90
4,ff794ed0-6ac1-4b80-8ab9-8bafd2856a35,22786,42,11,chg,90
5,1bfa73c2-3424-4a3a-bcd8-a95416fd9e27,22660,42,11,chg,90
6,956c2891-d7d8-4a25-93be-47b5b243f359,22714,42,11,chg,90
7,23f250ba-58a0-4957-b6ce-95dc0e9cddc1,22858,42,11,chg,90


In [13]:
[1, ] * 10

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [14]:
df = fs.get_all_features(format='df').copy(deep=True)

In [ ]:
data = df
data['voltage']

[array([3.4756, 3.4979, 3.5007, 3.5016, 3.5029, 3.5032, 3.5038, 3.5044,
        3.5047, 3.505 , 3.5053, 3.5057, 3.5063, 3.5066, 3.5069, 3.5072,
        3.5075, 3.5078, 3.5081, 3.5084, 3.5084, 3.5088, 3.5091, 3.5094,
        3.5094, 3.5097, 3.51  , 3.5103, 3.5103, 3.5109, 3.5112, 3.5984,
        3.6086, 3.6126, 3.6154, 3.6176, 3.6191, 3.621 , 3.6228, 3.6244,
        3.6256, 3.5168, 3.5084, 3.5053, 3.5035, 3.5016, 3.5007, 3.4998,
        3.4988, 3.4982, 3.4976, 3.497 , 3.4967, 3.4964, 3.4957, 3.4954,
        3.4945, 3.4942, 3.4939, 3.4936, 3.4933, 3.4929, 3.4926, 3.4923,
        3.492 , 3.492 , 3.4917, 3.4914, 3.4911, 3.4911, 3.4908, 3.4905,
        3.4902, 3.4902, 3.4898, 3.4895, 3.4895, 3.4892, 3.4892, 3.4889,
        3.4889, 3.4886, 3.4883, 3.4883, 3.4883, 3.488 , 3.488 , 3.4877,
        3.4877, 3.4874, 3.4874, 3.4871, 3.4871, 3.4864, 3.4864, 3.4864,
        3.4861, 3.4861, 3.4858, 3.4858, 3.4858]),
 array([3.4855, 3.4657, 3.4607, 3.4595, 3.4585, 3.4576, 3.457 , 3.4564,
        3.4561